In [2]:
data = 'https://archive.ics.uci.edu/static/public/222/bank+marketing.zip'

In [3]:
!wget $data

--2024-10-09 18:24:00--  https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘bank+marketing.zip’

bank+marketing.zip      [    <=>             ] 999.85K  1.04MB/s    in 0.9s    

2024-10-09 18:24:03 (1.04 MB/s) - ‘bank+marketing.zip’ saved [1023843]



In [18]:
import pandas as pd

In [54]:
df.dtypes

age           int64
job          object
marital      object
education    object
balance       int64
housing      object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y             int64
dtype: object

In [19]:
df = pd.read_csv('bank-full.csv', sep=';')

In [20]:
df = df[['age', 'job', 'marital', 'education', 'balance', 'housing',
       'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y']]

In [21]:
df.education.value_counts()

education
secondary    23202
tertiary     13301
primary       6851
unknown       1857
Name: count, dtype: int64

In [22]:
df.mode().T

,0
age,32
job,blue-collar
marital,married
education,secondary
balance,0
housing,yes
contact,cellular
day,20
month,may
duration,124


### Question 2

In [27]:
vars = ['age', 'balance', 'day', 'campaign', 'pdays', 'previous']

In [31]:
df[vars].corrwith(df.y).abs()

age         0.025155
balance     0.052838
day         0.028348
campaign    0.073172
pdays       0.103621
previous    0.093236
dtype: float64

In [42]:
v = ['age']
df[v].corrwith(df.balance).abs()

age    0.097783
dtype: float64

In [41]:
v = ['day']
df[v].corrwith(df.campaign).abs()

day    0.16249
dtype: float64

In [40]:
df[v].corrwith(df.pdays).abs()

pdays    1.0
dtype: float64

In [39]:
v=['pdays']
df[v].corrwith(df.previous).abs()

pdays    0.45482
dtype: float64

### Question 3

In [43]:
df.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,0


In [44]:
df.y = df.y.replace({'yes': 1, 'no':0})

In [45]:
df.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,0


In [46]:
from sklearn.model_selection import train_test_split

In [47]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [48]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [49]:
len(df_train), len(df_val), len(df_test)

(27126, 9042, 9043)

In [50]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [51]:
y_train = df_train.y.values
y_val = df_val.y.values
y_test = df_test.y.values

In [52]:
del df_train['y']
del df_val['y']
del df_test['y']

In [53]:
from sklearn.metrics import mutual_info_score

In [55]:
mutual_info_score(df_full_train.y, df_full_train.contact)

np.float64(0.014163614885003041)

In [57]:
mutual_info_score(df_full_train.y, df_full_train.education)

np.float64(0.0024578282035066477)

In [58]:
mutual_info_score(df_full_train.y, df_full_train.housing)

np.float64(0.00980003813867901)

In [59]:
mutual_info_score(df_full_train.y, df_full_train.poutcome)

np.float64(0.02925655626647966)